# Lights, Camera, ReAction


**ADD HOW INFO IS STORED IN META DATA**

This notebook builds a full **sitcom script generation pipeline** powered by **ReAct-based agents** — a powerful framework that combines **reasoning** (thinking through a problem) with **acting** (taking structured steps).

We start by **generating a sitcom concept** from creative keywords, then **outlining** the pilot episode scene-by-scene.  
**Scene 1** is generated directly from the outline to establish the world and tone.  
After that, each new scene is **scripted, reviewed, and improved** using specialized **ReAct agents** — a **Character Agent**, **Comedy Agent**, and **Environment Agent** — that simulate a real sitcom writers' room.

---

 **Benefits of using ReAct agents**:

- **More structured and transparent thinking:** Agents reason step-by-step before making edits.
- **Dynamic adaptation:** Agents flexibly plan the next creative moves based on scene history.
- **Better long-term coherence:** Scenes evolve logically, with tracked character growth, running jokes, emotional arcs, and worldbuilding.

---

After generation, each scene is **summarized and stored in a vector database**, enabling fast retrieval of scene metadata for future story planning.  
By combining **structured agent workflows** and **retrieval-augmented memory**, we bring sitcom worlds to life — one coherent, character-driven scene at a time.


In [1]:
!pip install datasets
!pip install sentence-transformers faiss-cpu

### Mounting Drive and Appending System

In [8]:
from google.colab import drive
import sys

drive.mount('/content/drive')
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# TO DELETE

!ls "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils"


agents		       __pycache__	  script_review.py  vector_db_utils.py
outline_generation.py  screen_writing.py  text_utils.py


In [10]:
from agents.character_agent import CharacterAgent
from agents.comedy_agent import ComedicAgent
from agents.environment_agents import EnvironmentReActAgent
from agents.scene_planner_agent import ScenePlannerAgent

from text_utils import (
    display_markdown_output,
    extract_scene,
    extract_title,
)

from outline_generation import (
    generate_sitcom_pitch,
    generate_pilot_episode_outline
)

from script_review import (
    validate_episode_outline,
)

from screen_writing import (
    generate_scene_1_script,
    generate_scene
)

from vector_db_utils import (
    summarize_scene,
    add_scene_to_vector_db
)

In [11]:
import numpy as np
import openai
from openai import OpenAI
from IPython.display import Markdown, display
import re
import time
from datetime import timedelta
from google.colab import userdata
import os
from sentence_transformers import SentenceTransformer
import faiss

In [12]:
from google.colab import userdata
import os

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

### Vector Database

This block initializes the sentence embedding model and sets up a FAISS index to store vector representations of scenes. The `all-MiniLM-L6-v2` model encodes scenes into fixed-size embeddings, and the FAISS index enables fast vector operations. The `scene_metadata` list stores contextual information linked to each scene embedding.


In [13]:
# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Vector store initialization
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

# To track metadata
scene_metadata = []


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 🎬 Generating the Sitcom Pitch

This step generates a sitcom concept using the `generate_sitcom_pitch` function. The function takes in a dictionary of categorized keywords (e.g., setting, characters, themes, tone/genre) to guide the generation process.

- The language model plays the role of a **professional screenwriter**.
- It produces a 1-paragraph pitch that includes a **title** and a **detailed premise**, highlighting the setting, main characters, their dynamics, and the overall tone of the show.
- Keywords help ground the pitch in user-defined ideas, ensuring thematic and narrative consistency.

In [14]:
keywords = {
    "setting": ["urban locksmith shop", "New York City"],
    "characters": ["ex-con protagonist", "estranged daughter", "quirky parole officer"],
    "themes": ["second chances", "odd couple dynamic"],
    "tone_genre": ["buddy comedy", "heartfelt absurdism"]
}


**TODO**

Make sure the title is quotation marks

In [15]:
# Generating Sitcom Pitch
sitcom_pitch = generate_sitcom_pitch(client, keywords)
display(Markdown(f"### Sitcom Concept:\n\n{sitcom_pitch}"))


### Sitcom Concept:

Title: "Key Changes"

"Key Changes" is a heartfelt yet absurdly comic sitcom set in an urban locksmith shop in the bustling heart of New York City. After serving time, ex-con 'Big Jimmy', with his solid build and soft heart, decides to leave his criminal past behind and open up a locksmith shop, hoping to unlock a new chapter in his life. When his estranged, sassy and street-smart daughter, 'Alexis', unexpectedly comes into his life and starts working at the shop, it's not just locks that get picked but their old wounds too. Adding to the chaos is Jimmy's quirkily zealous parole officer 'Stanley', who, seeking a side hustle, insists on becoming their unlikely business partner, despite knowing nothing about locksmithing. "Key Changes" explores themes of second chances and familial reconciliation with an odd couple dynamic, blending heartwarming moments with outrageous comedy, as this mismatched trio navigates running a business, rebuilding relationships, and the always unpredictable NYC customer base.

### 📝 Generating a Pilot Episode Outline

The next part is to generate an episode outline for the pilot of the show, which is done using the `generate_pilot_episode_outline` function. This function takes in the sitcom pitch and produces a structured scene-by-scene breakdown of the pilot episode, simulating the voice of a **professional sitcom writer** preparing a network pitch.

- Begins with a concise **Episode Concept** (1–2 sentences).
- Breaks the episode into approximately 20 **numbered scenes**, each with a **title in quotation marks** and a brief description.
- Each scene summary includes what happens, who is involved, and the intended tone (e.g., funny, awkward, heartfelt).

This outline provides the narrative foundation for generating full scene scripts in the next stage.


In [16]:
outline = generate_pilot_episode_outline(client, sitcom_pitch)
display(Markdown(f"### Sitcom Outline:\n\n{outline}"))

### Sitcom Outline:

Episode Concept: The pilot episode introduces the characters and the locksmith shop, with Big Jimmy trying to reconnect with his estranged daughter Alexis while starting a business. Meanwhile, Stanley turns up to become an unexpected business partner.

Scene 1: "Unlocking Opportunities" Big Jimmy, just released from prison, signs a lease for a locksmith shop and explains his new venture to his skeptical but supportive roommate, a funny and heartwarming scene.

Scene 2: "An Unexpected Surprise" Alexis unexpectedly enters the shop, revealing she is Jimmy's estranged daughter, an awkward and emotional scene.

Scene 3: "A Business Proposition" Stanley, Jimmy's parole officer, pops in for a visit and proposes becoming a partner in the locksmith business, a comedic and absurd scene.

Scene 4: "The First Customer" The trio handles their first customer, leading to hilarious misunderstandings about locksmithing.

Scene 5: "A Daughter's Distrust" Alexis confronts Jimmy about his past, a serious and touching scene that sheds light on their strained relationship.

Scene 6: "Locksmith 101" Stanley tries to learn the basics of locksmithing, leading to funny mishaps and pranks by Alexis.

Scene 7: "Stanley's Side Hustle" Stanley explains his motivation for wanting a side job, a comedic scene showing his bizarre reasons.

Scene 8: "Jimmy's Second Chance" Jimmy shares his hopes of turning his life around, a poignant and heartfelt scene.

Scene 9: "Father-Daughter Lock-Picking" Jimmy teaches Alexis how to pick a lock, leading to a mix of comedic and bonding moments.

Scene 10: "Breaking Down the Door" The trio responds to a lockout emergency, leading to hilarious chaos as they fumble through the job.

Scene 11: "Stanley's Slip-up" Stanley accidentally locks himself inside the shop's bathroom, resulting in a funny rescue mission. 

Scene 12: "Alexis' Big Ask" Alexis asks Jimmy why he never looked for her, an emotional scene that reveals more of their backstory.

Scene 13: "Stanley's Sales Pitch" Stanley attempts to sell locksmithing services to passersby, leading to comedic rejections.

Scene 14: "Locksmith Competition" The arrival of a rival locksmith introduces competitive tension, marked by humor and rivalry.

Scene 15: "Apologies and Keys" Jimmy sincerely apologizes to Alexis for his past mistakes, a touching and emotional scene.

Scene 16: "The Big Break-In" The trio tackles a complex job of a broken lock, leading to comedic sequences and triumph.

Scene 17: "A New Lock on Life" Jimmy and Alexis bond over their success on the job, a heartwarming father-daughter moment.

Scene 18: "Stanley's Progress Report" Stanley checks in on Jimmy's parole progress, a hilarious scene due to Stanley's unorthodox methods.

Scene 19: "Locks of Love" The trio celebrates their first successful day, sharing a heartfelt moment of camaraderie.

Scene 20: "Key to the Future" The episode ends with a promising outlook for the locksmith shop and Jimmy, Alexis, and Stanley's evolving relationships, a blend of comedy and hope.

### ✅ Episode Validation

To ensure that the sitcom pilot outline is coherent and aligns with the original pitch, we use the `validate_episode_outline` function. This function simulates the voice of a **veteran sitcom writer** who evaluates whether the episode outline makes narrative sense and fits the tone and premise.

- Checks for overall **coherence**, including consistency of tone, logical character progression, and structural viability.
- Returns a clear **"Yes" or "No"** on coherence, along with 2–4 bullet points of reasoning.
- If the episode is not coherent, it also provides concrete suggestions for improvement.

This validation step helps maintain narrative quality before moving into full scene scripting.


**TODO**

- Make sure it can re-generate the script
- Delte timedelta

In [17]:
validation = validate_episode_outline(client, sitcom_pitch, outline)
display(Markdown(f"### Coherence Evaluation:\n\n{validation}"))

### Coherence Evaluation:

Coherence: Yes

Reasoning:
- The episode outline fits the premise of the sitcom pitch, introducing the main characters and the premise of the locksmith shop. The characters' relationships and dynamics are established, and the tone alternates between comedy and heartfelt moments, as described in the pitch.
- The character progression is logical, with Big Jimmy trying to reconcile with his estranged daughter Alexis and start a new life, Alexis trying to trust her father, and Stanley trying to fit into the business despite knowing nothing about locksmithing.
- The plot progression is also logical, with the trio handling their first customers, dealing with a rival locksmith, and eventually celebrating their first successful day.
- The scenes are varied and engaging, mixing comedic situations with emotional moments, which is essential for a sitcom. The setting in NYC and the locksmith shop is used to its full potential, providing a unique backdrop for the comedy and drama.

## Scene 1 Generation

In [18]:
scene_1_desc = extract_scene(outline, 1)
display(Markdown(scene_1_desc))

Scene 1: "Unlocking Opportunities" Big Jimmy, just released from prison, signs a lease for a locksmith shop and explains his new venture to his skeptical but supportive roommate, a funny and heartwarming scene.

In [19]:
# Extracting title
sitcom_title = extract_title(sitcom_pitch)
print(sitcom_title)

Key Changes


In [20]:
scene_1_script = generate_scene_1_script(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_1_desc,
    rag_context=None,
    line_target=(55, 75)
)

In [21]:
display(Markdown(scene_1_script))


INT. APARTMENT LIVING ROOM - DAY

Big JIMMY, a burly ex-con with a heart of gold, is sitting on a dingy couch. He is holding a lease agreement, grinning. His roommate, LARRY, a wiry fast-talker with a sarcastic wit, enters the room.

[LAUGH TRACK]

JIMMY
Larry, I've done it!

LARRY
If "it" means you finally cleaned the bathroom, then we're both winners today.

[LAUGH TRACK]

JIMMY
No, Larry. I signed the lease on a shop today.

LARRY
(raises an eyebrow)
A shop? What kind of shop?

JIMMY
(grinning)
A locksmith shop!

LARRY
(rolls eyes)
Of course. A locksmith shop. Why didn't I think of that?

[LAUGH TRACK]

JIMMY
I know it sounds a bit out there, but it’s a legitimate business! Honest work!

LARRY
(skeptical)
Yeah, and a great cover for a former cat burglar.

[LAUGH TRACK]

JIMMY
(serious)
Larry, I'm trying to turn over a new key here.

[LAUGH TRACK]

LARRY
(chuckles)
You mean leaf?

JIMMY
No, key. It’s a locksmith joke.

[LAUGH TRACK]

LARRY
(smirks)
Well, Jimmy, I hope this "key" change of yours doesn't "lock" you back in prison.

[LAUGH TRACK]

JIMMY
(smiles)
Nah, I'm past that. I'm unlocking new opportunities.

LARRY
(sighs)
Well, if you're happy, I’m happy. Besides, I've always wanted to say I live with a professional lock picker.

[LAUGH TRACK]

JIMMY
(locksmith)
Locksmith, Larry.

LARRY
(grins)
Yeah, yeah.

They both laugh and Jimmy starts reading the lease again. Larry goes to the kitchen, shaking his head.

FADE OUT.

### 🗂️ Adding Metadata to the Vector Database

To extract structured information from each generated sitcom scene, we use th `summarize_scene` function. This function simulates the head writer of the show and analyzes the scene script to produce a concise summary along with key metadata.

- **Summary**: A 100–150 word overview of the scene’s key actions, character dynamics, and notable moments.
- **Characters**: A list of characters who appear or speak in the scene.
- **Location**: The main setting of the scene, if clearly identifiable (e.g., “locksmith shop,” “apartment hallway”).
- **Recurring Joke**: Any running gag or callback featured in the scene.
- **Emotional Tone**: A 1–2 word description of the scene’s mood (e.g., “chaotic,” “hopeful,” “awkward”).

The metadata is returned as a dictionary and appended to the `scene_metadata` list using the `add_scene_to_vector_db` function. This step is crucial: instead of supplying the full script to the next generation phase, we pass a concise summary and key elements to guide smoother, context-aware scene generation.


In [22]:
scene_1_summary = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_1_script
)

In [23]:
# Adds scene to the vector database
add_scene_to_vector_db(
    scene_1_summary,
    full_script=scene_1_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=scene_metadata
)


In [24]:
print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(scene_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 1 


Scene 1
Summary: In the scene, Jimmy, a burly ex-con, excitedly reveals to his sarcastic roommate, Larry, that he has signed a lease for a locksmith shop. Larry initially responds with skepticism, joking about Jimmy's past as a cat burglar, but ultimately expresses support for Jimmy's new venture. The scene includes a running joke about Jimmy's past and his new profession, with several puns related to keys and locks. The scene ends with Jimmy reading the lease and Larry heading to the kitchen, both in good spirits.
Characters: ['Jimmy', 'Larry']
Location: Apartment Living Room
Recurring Joke: The recurring joke in this scene is the puns related to keys and locks, referencing Jimmy's past as a cat burglar and his new profession as a locksmith.
Emotional Tone: Amused


## 🎬 Scene 2 Generation

The next part is to generate Scene 2. This is one of the most important scenes in the script, as it’s where the ReAct (Reasoning + Acting) system comes into play.

Metadata from Scene 1 is retrieved and interpreted by the `CharacterAgent`, `ComedyAgent`, and `EnvironmentAgent`. Each agent reasons over the metadata independently and provides creative suggestions based on its area of expertise—such as character consistency, comedic dynamics, or environmental context.

These agent responses are then passed to the `ScenePlannerAgent`, which integrates their inputs and plans the next scene accordingly. This collaborative reasoning-and-acting process ensures that each scene builds logically and creatively on what came before.

The advantage of using ReAct-style agents over purely role-based agents is that each agent not only plays a role but also actively reasons and adapts based on prior context—making the generation more dynamic, context-aware, and narratively coherent.


In [25]:
scene_2_desc = extract_scene(outline, 2)
print(scene_2_desc)

Scene 2: "An Unexpected Surprise" Alexis unexpectedly enters the shop, revealing she is Jimmy's estranged daughter, an awkward and emotional scene.


### 🎭 Character Agent

The first step is to analyze the characters using the `CharacterAgent`. This agent is responsible for tracking which characters appear in each scene, evaluating their consistency with past behavior, and recommending meaningful interactions to advance the story.

The `CharacterAgent` operates using a ReAct workflow:
- **Think**: Identifies which characters are present in the current scene and compares them to recent scenes.
- **Act**: Retrieves character histories by analyzing summaries from previous episodes.
- **Observe**: Checks whether characters behave consistently with their established traits and arcs.
- **Recommend**: Suggests specific character interactions that align with the narrative or help resolve inconsistencies.

Each step simulates the judgment of real writers' room roles — from a Writers' Assistant and Script Supervisor to the Head Writer and Co-Executive Producer — ensuring character development is grounded, coherent, and creatively generative.


In [38]:
import importlib
import agents.character_agent
importlib.reload(agents.character_agent)
from agents.character_agent import CharacterAgent


In [39]:
'''
MOVE TO SCRIPT
'''

from typing import Tuple, Dict, List

def evaluate_character_agent_scene(
    agent: CharacterAgent,
    scene_description: str,
    scene_number: int
) -> Tuple[Dict[str, Dict], bool, str, str, List[str]]:
    """
    Runs the CharacterAgent on a given scene and prints a cleaned summary of results:
    - Consistency verdict
    - Short explanation
    - 2 recommended interactions
    - Agent's internal thoughts

    Args:
        agent (CharacterAgent): The CharacterAgent instance.
        scene_description (str): The current scene's description.
        scene_number (int): The scene number being evaluated.

    Returns:
        Tuple of:
            - character_histories (dict)
            - is_consistent (bool)
            - explanation (str)
            - recommendations (str)
            - internal_thoughts (list of str)
    """
    print(f"🔁 Agent will consider the last {agent.num_scenes} scene(s) for context.\n")

    character_histories, is_consistent, explanation, recommendations, thoughts = agent.run(
        scene_description=scene_description,
        scene_number=scene_number
    )

    # Extract short explanation only
    short_explanation = ""
    for line in explanation.split("\n"):
        if line.strip().startswith("2. Short Explanation Why:"):
            short_explanation = line.strip()
            break

    # Only keep the two interaction suggestions (strip label)
    clean_recommendations = "\n".join([
        line for line in recommendations.split("\n")
        if line.strip().startswith("1.") or line.strip().startswith("2.")
    ])

    print(f"Scene {scene_number} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}\n")
    print("Explanation:")
    print(short_explanation if short_explanation else explanation.strip())
    print("\nInteraction Improvement Recommendations:")
    print(clean_recommendations.strip())
    print("\nAgent's Internal Thoughts:")
    for thought in thoughts:
        print("-", thought)

    return character_histories, is_consistent, explanation, recommendations, thoughts


In [40]:
character_agent = CharacterAgent(
    client=client,
    vector_metadata=scene_metadata,
    num_scenes=1
)

evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_2_desc,
    scene_number=2,
);


🔁 Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Jimmy's optimistic and resilient personality is consistent with his reaction to Alexis's unexpected arrival. His good-humored and tolerant nature is also consistent with his ability to handle the awkward and emotional situation. Alexis's boldness is consistent with her decision to confront Jimmy. However, more information is needed to fully assess the consistency of Alexis's character.

Interaction Improvement Recommendations:
1. Jimmy, upon seeing Alexis, unconsciously fiddles with a lock and key he has been working on, using humor to diffuse the tension by joking about how she "has a knack for picking locks, just like her old man." — (In Scene 1, Jimmy is revealed to have a past as a cat burglar, which he now uses as a source of humor. His joke serves to lighten the situation while acknowledging their shared

In [37]:

# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=scene_metadata,
    num_scenes=1  # optional — defaults to 1
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

print(f"Scene {2} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)



AttributeError: 'CharacterAgent' object has no attribute 'run'

In [ ]:
stop

### Comedy Agent

In [36]:
# Initialize the agent
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=scene_metadata,
    num_scenes=1  # optional — defaults to 3
)

# Run the full ReAct loop
context, is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

# Print formatted output
print(f"Scene {2} — Comedic Tone: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")

print("\nComedic Tone Analysis:\n", analysis_text)

print("\nComedic Improvement Recommendations:\n", com_recommendations)

print("\nComedic Agent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


KeyboardInterrupt: 

### Enviroment Agent



In [ ]:
# Initialize the environment agent
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=scene_metadata,  # prior scene metadata
    num_scenes=1  # how many scenes to look back
)

# Run the ReAct cycle for scene 2
context, is_consistent, explanation, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_2_desc,
    scene_number=2
)

# Display the results
print(f"Scene {2} — Environment Transition: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")

print("\nExplanation:\n", explanation)

print("\nEnvironment Detail Suggestions:\n", environment_details_suggestions)

print("\nEnvironment Agent's Internal Thoughts:")
for thought in env_thoughts:
    print("-", thought)


Scene 2 — Environment Transition: ✅ Consistent

Explanation:
 Short Explanation: The transition is logical as the characters are still in the same location, Chuckles' Shop. The name might be slightly different, but it's clear that it's the same place. There's no need for a setup or linking action in this case.

Environment Detail Suggestions:
 Environment Details Suggestions:
- As Samantha's nerves reach a new high, she starts fidgeting with a small wind-up toy clown that Chuckles' Shop has on the counter. The clown toy suddenly springs to life, flailing its tiny arms and bobbing its head, which only adds to the awkwardness of the situation.
- The shop also has a novelty doorbell that plays a silly, upbeat jingle every time someone enters or exits. The jingle loudly interrupts a particularly tense moment, causing the characters to jump and then laugh awkwardly to ease the tension.

Environment Agent's Internal Thoughts:
- Think: Analyzed environment features for Scene 2.
- Observe: Sce

In [ ]:
stop

NameError: name 'stop' is not defined

### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_2_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=2
)

In [ ]:
print(scene_2_plan)

### Generating Scene 2 Script

**Make sure the physical enviroment is conssitent over the whole scene**

**Define what a scene is to the agent**

In [ ]:
scene_2_script = generate_scene(
    client=client,
    scene_plan=scene_2_plan,
    scene_number= 2
)

print(scene_2_script)

### Adding Metadata to Vector Database

In [ ]:
scene_metadata_2 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_2_script
)


In [ ]:
# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_2,
    full_script=scene_2_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


In [ ]:
print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


## Scene 3 Generation

In [ ]:
scene_3_desc = extract_scene(outline, 3)
print(scene_3_desc)

### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=2
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

# Outputs
print(f"Scene {3} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=2  # or 2, 3, 5 — however many past scenes you want it to look at
)

In [ ]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_3_desc,
    scene_number=3
)

In [ ]:
print(environment_details_suggestions)

### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_3_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=3
)

In [ ]:
print(scene_3_plan)

### Generating Scene 3 Script

In [ ]:
scene_3_script = generate_scene(
    client=client,
    scene_plan=scene_3_plan,
    scene_number= 3
)

print(scene_3_script)

### Adding Metadata to Vector Database

In [ ]:
'''
MAKE FUNCTION!!!
'''

scene_metadata_3 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_3_script
)


# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_3,
    full_script=scene_3_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


## Scene 4 Generation

In [ ]:
scene_4_desc = extract_scene(outline, 4)
print(scene_4_desc)

### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

# Outputs
print(f"Scene {4} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=3
)

In [ ]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

In [ ]:
print(environment_details_suggestions)

### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_4_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=4
)

In [ ]:
print(scene_4_plan)

### Generating Scene 4 Script

In [ ]:
scene_4_script = generate_scene(
    client=client,
    scene_plan=scene_4_plan,
    scene_number= 4
)

print(scene_4_script)

## Scene 5 Generation

In [ ]:
scene_5_desc = extract_scene(outline, 5)
print(scene_5_desc)

### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

# Outputs
print(f"Scene {5} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=3
)

environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

print(environment_details_suggestions)

### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_5_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=5
)

print(scene_5_plan)

### Generating Scene 5 Script

In [ ]:
scene_5_script = generate_scene(
    client=client,
    scene_plan=scene_5_plan,
    scene_number= 5
)

print(scene_5_script)

## Combining Scripts



In [ ]:
# List your scene scripts in order
scene_scripts = [
    scene_1_script,
    scene_2_script,
    scene_3_script,
    scene_4_script,
    scene_5_script
]

# Combine with optional scene headers
full_episode_script = "\n\n".join([
    f"### Scene {i+1} ###\n{script.strip()}"
    for i, script in enumerate(scene_scripts)
])

print(full_episode_script)